In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from scipy.special import expit
from sklearn.metrics import accuracy_score

In [ ]:
import pandas as pd

url = "https://github.com/propublica/compas-analysis/raw/master/compas-scores-two-years.csv"
compas = pd.read_csv(url)


compas = compas[(compas["race"]=="African-American")|(compas["race"]=="Caucasian")]
compas = pd.get_dummies(compas, columns = ["race"], drop_first = True)
compas

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,juv_fel_count,...,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid,race_Caucasian
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,0,...,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1,0
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,0,...,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1,0
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,0,...,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0,0
6,8,edward riddle,edward,riddle,2014-02-19,Male,1974-07-23,41,25 - 45,0,...,Low,2014-02-19,2014-03-31,2014-04-18,14,5,40,1,1,1
8,10,elizabeth thieme,elizabeth,thieme,2014-03-16,Female,1976-06-03,39,25 - 45,0,...,Low,2014-03-16,2014-03-15,2014-03-18,0,2,747,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7207,10994,jarred payne,jarred,payne,2014-05-10,Male,1985-07-31,30,25 - 45,0,...,Low,2014-05-10,2015-10-22,2015-10-22,0,0,529,1,1,0
7208,10995,raheem smith,raheem,smith,2013-10-20,Male,1995-06-28,20,Less than 25,0,...,High,2013-10-20,2014-04-07,2014-04-27,0,0,169,0,0,0
7209,10996,steven butler,steven,butler,2013-11-23,Male,1992-07-17,23,Less than 25,0,...,Medium,2013-11-23,2013-11-22,2013-11-24,0,1,860,0,0,0
7210,10997,malcolm simmons,malcolm,simmons,2014-02-01,Male,1993-03-25,23,Less than 25,0,...,Medium,2014-02-01,2014-01-31,2014-02-02,0,1,790,0,0,0


In [ ]:
X = compas[['age', 'sex', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count', 'c_charge_degree', 'days_b_screening_arrest', 'is_recid', 'is_violent_recid']]
Y = compas['two_year_recid']
S = compas['race_Caucasian']

In [ ]:
X = X.fillna(0)

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6150 entries, 1 to 7212
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      6150 non-null   int64  
 1   sex                      6150 non-null   object 
 2   juv_fel_count            6150 non-null   int64  
 3   juv_misd_count           6150 non-null   int64  
 4   juv_other_count          6150 non-null   int64  
 5   priors_count             6150 non-null   int64  
 6   c_charge_degree          6150 non-null   object 
 7   days_b_screening_arrest  6150 non-null   float64
 8   is_recid                 6150 non-null   int64  
 9   is_violent_recid         6150 non-null   int64  
dtypes: float64(1), int64(7), object(2)
memory usage: 528.5+ KB


In [ ]:
X.describe()

,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,is_recid,is_violent_recid
count,6150.000000,6150.000000,6150.000000,6150.000000,6150.000000,6150.000000,6150.000000,6150.000000
mean,34.730244,0.070732,0.098699,0.117886,3.699837,2.938537,0.497724,0.116585
std,11.907074,0.436621,0.504829,0.517938,5.028782,72.023454,0.500035,0.320952
min,18.000000,0.000000,0.000000,0.000000,0.000000,-414.000000,0.000000,0.000000
25%,25.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000
50%,31.000000,0.000000,0.000000,0.000000,2.000000,-1.000000,0.000000,0.000000
75%,42.000000,0.000000,0.000000,0.000000,5.000000,0.000000,1.000000,0.000000
max,83.000000,10.000000,13.000000,17.000000,38.000000,1057.000000,1.000000,1.000000


In [ ]:
X = pd.get_dummies(X, columns=['sex','c_charge_degree'], drop_first = True)
X[['is_recid','is_violent_recid']] = X[['is_recid','is_violent_recid']].astype("uint8")
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6150 entries, 1 to 7212
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      6150 non-null   int64  
 1   juv_fel_count            6150 non-null   int64  
 2   juv_misd_count           6150 non-null   int64  
 3   juv_other_count          6150 non-null   int64  
 4   priors_count             6150 non-null   int64  
 5   days_b_screening_arrest  6150 non-null   float64
 6   is_recid                 6150 non-null   uint8  
 7   is_violent_recid         6150 non-null   uint8  
 8   sex_Male                 6150 non-null   uint8  
 9   c_charge_degree_M        6150 non-null   uint8  
dtypes: float64(1), int64(5), uint8(4)
memory usage: 360.4 KB


In [ ]:
non_uint8_columns = X.select_dtypes(exclude='uint8').columns
scaler = StandardScaler()
X[non_uint8_columns] = scaler.fit_transform(X[non_uint8_columns])
X

,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,is_recid,is_violent_recid,sex_Male,c_charge_degree_M
1,-0.061334,-0.162011,-0.195526,-0.227625,-0.735792,-0.054689,1,1,1,0
2,-0.901239,-0.162011,-0.195526,1.703263,0.059694,-0.054689,1,0,1,0
3,-0.985229,-0.162011,1.785502,-0.227625,-0.536921,-0.040803,0,0,1,0
6,0.526600,-0.162011,-0.195526,-0.227625,2.048409,-0.054689,1,0,1,0
8,0.358619,-0.162011,-0.195526,-0.227625,-0.735792,-0.054689,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
7207,-0.397296,-0.162011,-0.195526,-0.227625,-0.735792,-0.054689,1,0,1,1
7208,-1.237201,-0.162011,-0.195526,-0.227625,-0.735792,-0.054689,0,0,1,0
7209,-0.985229,-0.162011,-0.195526,-0.227625,-0.735792,-0.054689,0,0,1,0
7210,-0.985229,-0.162011,-0.195526,-0.227625,-0.735792,-0.054689,0,0,1,0


In [ ]:
#Get gradient for gradient descent
def gradient(X_Y_S_train, weight_s, eta, lambda_l2):

    #Part1: Calculating gradient of loss function

    #Gradient of lnM is gradient of binary cross entropy loss
    #For each xi, gradient of loss is (y_pred-y)*xi
    def gradient_lnM(row):
        xi = row[range(len(X.columns))]
        w_s = weight_s[row[len(X.columns)]]
        y = row[-1]
        y_hat = expit(np.matmul(xi, w_s.reshape(-1,1)))
        return (y_hat-y)*xi

    gradient_loss = np.apply_along_axis(gradient_lnM, axis=1, arr = np.array(X_Y_S_train))
    gradient_cost = np.sum(gradient_loss, axis = 0)

    #Part2: Calculating gradient of prejudice remover regularizer

    #M function
    def M_func_numpy(row):
        xi = row[range(len(X.columns))]
        w_s = weight_s[row[len(X.columns)]].reshape(-1,1)
        y = row[-1]
        sigmoid_input = np.matmul(xi, w_s)
        sigmoid_output = expit(sigmoid_input)
        return y*sigmoid_output+(1-y)*(1-sigmoid_output)

    #Gradient of M function
    def M_gradient(row):
        xi = row[range(len(X.columns))]
        w_s = weight_s[row[len(X.columns)]]
        y = row[-1]
        y_hat = expit(np.matmul(xi, w_s.reshape(-1,1)))
        return (2*y-1)*y_hat*(1-y_hat)*xi

    R_pr_gradient = 0
    for s in list(S.unique()):
        filter_s = X_Y_S_train[X_Y_S_train[S.name]==s]
        for y in [0,1]:
            #Calculate P(y)
            df1 = X_Y_S_train[X_Y_S_train[Y.name]==y]
            M1 = np.apply_along_axis(M_func_numpy, axis=1, arr = np.array(df1))
            prob_y = np.sum(M1)/len(X_Y_S_train)
            #Calculate P(y|s)
            df2 = X_Y_S_train[(X_Y_S_train[S.name]==s)&(X_Y_S_train[Y.name]==y)]
            M2 = np.apply_along_axis(M_func_numpy, axis=1, arr = np.array(df2))
            prob_y_s = np.sum(M2)/len(filter_s)
            #Calculate gradient of prejudice remover regularizer
            #Since the gradient equation is too long, we divide it into 2 parts: p1 gradient and p2 gradient
            p1_gradient = np.apply_along_axis(M_gradient, axis=1, arr = np.array(df2))*np.log(prob_y_s/prob_y)
            #Since functions of P(y) and P(y|s) are based on M function,
            #when we calculate their corresponding gradients, we could calculate based on gradient of M function
            gradient_prob_y_s = np.sum(np.apply_along_axis(M_gradient, axis=1, arr = np.array(df2)), axis = 0)/len(filter_s)
            gradient_prob_y = np.sum(np.apply_along_axis(M_gradient, axis=1, arr = np.array(df1)), axis = 0)/len(X_Y_S_train)
            p2_gradient = M2*(prob_y/prob_y_s)*((gradient_prob_y_s*prob_y-prob_y_s*gradient_prob_y)/(prob_y**2))
            R_pr_gradient += np.sum(p1_gradient+p2_gradient, axis = 0)

    #Part3: Calculating gradient of L2 regularizer

    #Gradient of (||w||_2)^2 is 2wi
    l2_gradient = 0
    for s in list(S.unique()):
        l2_gradient += lambda_l2*weight_s[s]

    #Add up Loss function+prejudice remover regularizer+l2 regularizer to get final gradient
    return gradient_cost+eta*R_pr_gradient+l2_gradient

#Fit the model with train data and parameters
def fit_model(X_train, y_train, eta, lambda_l2, learning_rate, num_iterations):
    X_Y_S_train = pd.concat([X_train, y_train], axis=1, ignore_index=False)
    #Set initial w_s
    weight_s = dict()
    for s in list(S.unique()):
        model = LogisticRegression()
        model.fit(X_Y_S_train[X_Y_S_train[S.name]==s][X.columns], X_Y_S_train[X_Y_S_train[S.name]==s][Y.name])
        weight_s[s] = model.coef_
    #Update w_s in specified number of iterations
    for i in range(num_iterations):
        gradient_vec = gradient(X_Y_S_train, weight_s, eta, lambda_l2)
        for s in list(S.unique()):
            weight_s[s] -= learning_rate * gradient_vec
    return weight_s

#Predict y
def predict(X_test, weight_s):
    #Calculate probabilities and assign 1 or 0 to y_pred
    def y_predict(row):
        xi = row[range(len(X.columns))]
        w_s = weight_s[row[len(X.columns)]].reshape(-1,1)
        sigmoid_input = np.matmul(xi, w_s)
        sigmoid_output = expit(sigmoid_input)
        if sigmoid_output >= 0.5:
            return 1
        else:
            return 0
    return pd.Series(np.apply_along_axis(y_predict, axis=1,  arr = np.array(X_test)))

In [ ]:
X_S = pd.concat([X, S], axis=1, ignore_index=False)
X_train, X_test, y_train, y_test = train_test_split(X_S, Y, test_size=0.35, stratify=Y, random_state=42)
model_Before_regularizer = LogisticRegression()
model_Before_regularizer.fit(X_train, y_train)
predictions = model_Before_regularizer.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.9670227589410125


In [ ]:
new_data = X_test[X_test["race_Caucasian"]==0].iloc[:,:11]
predictions_A = model_Before_regularizer.predict(new_data)
acc_A_BR = accuracy_score(y_test[X_test[X_test["race_Caucasian"]==0].index],predictions_A)
print(f'African American Accuracy: {acc_A_BR}')

African American Accuracy: 0.9653270291568163


In [ ]:
new_data = X_test[X_test["race_Caucasian"]==1].iloc[:,:11]
predictions_C = model_Before_regularizer.predict(new_data)
acc_C_BR = accuracy_score(y_test[X_test[X_test["race_Caucasian"]==1].index],predictions_C)
print(f'Caucasian Accuracy: {acc_C_BR}')

Caucasian Accuracy: 0.9694570135746606


In [ ]:
print("Test calibration score: ", abs(acc_C_BR - acc_A_BR))

Test calibration score:  0.00412998441784429


In [ ]:
X_S = pd.concat([X, S], axis=1, ignore_index=False)
X_train, X_test, y_train, y_test = train_test_split(X_S, Y, test_size=0.35, stratify=Y, random_state=42)
optimal_weights = fit_model(X_train, y_train, 1, 1, 0.01, 10)
y_pred_test = predict(X_test, optimal_weights)
test_accuracy = accuracy_score(y_test, y_pred_test)
y_pred_train = predict(X_train, optimal_weights)
train_accuracy = accuracy_score(y_train, y_pred_train)
print(f'Test Accuracy: {test_accuracy}')
print(f'Train Accuracy: {train_accuracy}')

Test Accuracy: 0.9465861588481189
Train Accuracy: 0.9549662246685013


In [ ]:
new_data = X_test[X_test["race_Caucasian"]==0].iloc[:,:10]
log_odds_series = new_data.apply(lambda row: np.dot(optimal_weights[0], row), axis=1)
probability_series = 1 / (1 + np.exp(-log_odds_series.astype(float)))
classification_series = (probability_series > 0.5).astype(int)
acc_A = accuracy_score(y_test[X_test[X_test["race_Caucasian"]==0].index],classification_series)
print(f'African American Accuracy: {acc_A}')

African American Accuracy: 0.9432624113475178


In [ ]:
new_data = X_test[X_test["race_Caucasian"]==1].iloc[:,:10]
log_odds_series = new_data.apply(lambda row: np.dot(optimal_weights[1], row), axis=1)
probability_series = 1 / (1 + np.exp(-log_odds_series.astype(float)))
classification_series = (probability_series > 0.5).astype(int)
acc_C = accuracy_score(y_test[X_test[X_test["race_Caucasian"]==1].index],classification_series)
print(f'Caucasian Accuracy: {acc_C}')

Caucasian Accuracy: 0.9513574660633484


In [ ]:
print("Test calibration score: ", abs(acc_C - acc_A))

Test calibration score:  0.008095054715830607
